# Generative models exercise

## Introduction

This exercise is based on a normalizing flow exercise designed by T.Quadfasel, M.Sommerhalder and S.Diefenbacher, https://github.com/uhh-pd-ml/flow-exercise

In order to run the exercise a special environment is needed. In a terminal run the following 3 lines:




```
module load python

source activate /global/common/software/trn016/python_envs/ml4fp2025Day4

python -m ipykernel install --user --name training_env_pytorch --display-name "ML4FP2025_Day4"
```
Then swap the kernel of this notebook to 'ML4FP2025_Day4'


## Two Moons Data Set

One common example used for benchmarking is the so-called 'two moon' dataset, consisting of two interlocking half circles. On the one hand, it tests the generative model's ability to replicate a complex structure, on the other hand, this data set also features two separate subsets, which our generative model will have to keep separate as well. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# we will get the dataset from scikit-learn
import sklearn.datasets as datasets
from sklearn import cluster, datasets, mixture
from sklearn.preprocessing import StandardScaler

# for two moons dataset


# define normalization function
def normalize_moons(in_data):
    max_val = np.max(in_data, keepdims=True, axis=0)
    min_val = np.min(in_data, keepdims=True, axis=0)

    new_data = (in_data - min_val) / (max_val - min_val)
    mask = np.prod(((new_data < 1) & (new_data > 0)), axis=1, dtype=bool)
    new_data = new_data[mask]
    return new_data, mask


n = 10000
X_moons, _ = datasets.make_moons(n_samples=n, noise=0.05)
X_moons, _ = normalize_moons(X_moons)

plt.scatter(X_moons[:, 0], X_moons[:, 1], color='black', marker='.', linewidth=0)
plt.savefig("./original_two_moons.pdf")
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.show()
plt.close()

## PyTorch

In our next step, we import the PyTorch package, which we will be using as our machine learning backend. We also implement a generic fully connected neural network class, which we will be using in the following generative examples.

In [ ]:
# our usual PyTorch functionality
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F


cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")


# General purpose dense net
class DenseNet(torch.nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, n_layers, activation_fn=F.relu, last_activation_fn=None):
        super(DenseNet, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.layers = nn.ModuleList()
        self.layers.append(torch.nn.Linear(self.input_dim, self.hidden_dim))

        for i in range(self.n_layers):
            self.layers.append(torch.nn.Linear(self.hidden_dim, self.hidden_dim))

        self.layers.append(torch.nn.Linear(self.hidden_dim, self.output_dim))

        self.activation_fn = activation_fn
        self.last_activation_fn = last_activation_fn

    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            x = self.activation_fn(layer(x))
        x = self.layers[-1](x)
        if self.last_activation_fn is not None:
            x = self.last_activation_fn(x)
        return x

## 1) Variational AutoEncoder

The Variational Autoencoder implementation shown here explicitly defines the encoder and decoder models as two separate dense networks. The Endocer input and Decoder output have to match the dimensionality of the data set. The  Endocer output and Decoder input define the latent space their size is arbitrary, however, they need to be consistent such that the Encoder output is twice as large as the Decoder input. This is due to the fact in order to produce the Gaussian latent space, the Encoder needs to output a width and a mean for each Decoder input.

In principle, this specific VAE is a Beta-VAE, as it has an explicit scaling parameter between the MSE reconstruction loss and the KLD latent regularization loss. However, for a beta parameter of 1, this setup is identical to a standard VAE. 

**Task: The VEA implementation below is functional, but far from optimal. Run the subsequent two cells, and observe the Two Moons samples produced by the generator. Then modify some of the marked parameters and observe if the result improves. Especially the Beta parameter is interesting to vary**


In [ ]:
#######################################
### Modifiy network parameters here ###
EncoderNet = DenseNet(input_dim=2, output_dim=4, hidden_dim=32, n_layers=4).to(device)
DecoderNet = DenseNet(input_dim=2, output_dim=2, hidden_dim=32, n_layers=4).to(device)
#######################################


EncoderOpt = optim.Adam(EncoderNet.parameters(), lr=1e-5)
DecoderOpt = optim.Adam(DecoderNet.parameters(), lr=1e-5)

#######################################
### modify training parameters here ###
batch_size = 200
epochs = 100
#######################################

###################################
### modify loss parameters here ###
beta = 1
###################################

max_batches = int(X_moons.shape[0] / batch_size)

MSEloss = nn.MSELoss()
for ep in range(epochs):
    for i_batch in range(max_batches):
        EncoderOpt.zero_grad()
        DecoderOpt.zero_grad()

        # select the current batch from the dataset
        x_real = X_moons[i_batch * batch_size : (i_batch + 1) * batch_size]
        x_real = torch.tensor(x_real, device=device).float()

        latent = EncoderNet(x_real)
        mu = latent[:, ::2]
        log_var = latent[:, 1::2]

        KLD = torch.mean(-0.5 * torch.sum(1 + log_var - mu**2 - log_var.exp(), dim=1), dim=0)

        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std, device=device)
        reparameterized = eps * std + mu

        x_recon = DecoderNet(reparameterized)

        MSE = MSEloss(x_real, x_recon)

        loss = KLD * beta + MSE
        loss.backward()

        EncoderOpt.step()
        DecoderOpt.step()

    if ep % 10 == 0:
        print(ep)

In [ ]:
# Let's use the fitted distribution and sample from it
with torch.no_grad():
    noise = torch.randn((1000, 2), device=device).float()
    samples = DecoderNet(noise).cpu().numpy()


# plot the results
plt.scatter(X_moons[:, 0], X_moons[:, 1], color='black', marker='.', linewidth=0)
plt.scatter(samples[:, 0], samples[:, 1], color='darkblue', marker='o', linewidth=0)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.savefig("./sampled_two_moons.pdf")
plt.show()
plt.close()

# 2 Diffusion Models

## 2a.) Introduction

Much like normalizing flows, diffusion models are designed to learn the inverse of mapping a data point to random noise. The math underpinning diffusion models is ironically significantly more complex than the math required to train and run a diffusion model. 

Therefore, this section will focus on how to train a basic diffusion model, without performing the first principle derivations.

The fundamental idea behind diffusion models is to take a data point, repeatedly apply a Gaussian noise kernel to it, and then learn to undo that Gaussian smearing. In practice, repeatedly applying smearing is done using a Gaussian kernel with properties derived from the noise schedule beta. 
This emulates the effect of multiple smearings and broadly follows the original denoising diffusion papers, e.g. https://arxiv.org/abs/2006.11239

In [ ]:

timesteps = 10
beta = torch.linspace(start=1e-4, end=1e-1, steps=timesteps).to(device)

# define alpha for forward diffusion kernel
alpha = 1 - beta
alpha_bar = torch.cumprod(alpha, dim=0)


First, let us plot the noise function to see how the data looks, with added noise, after a given timestep:


In [ ]:
for step in range(timesteps-1, -1, -1):
    
    sample = torch.tensor(X_moons, device=device).float()
    noise = torch.randn_like(sample) 

    noisy_sample = (sample * torch.sqrt(alpha_bar[step]) + 
                    noise * torch.sqrt(1-alpha_bar[step]))
    
    samples = noisy_sample.cpu().numpy()
   
    plt.scatter(samples[:, 0], samples[:, 1], color='darkblue', marker='o', linewidth=0)
    plt.scatter(X_moons[:, 0], X_moons[:, 1], color='black', marker='.', linewidth=0)
    plt.savefig("./sampled_two_moons.pdf")
    plt.show()
    plt.close()


In order to allow the denoising model to reduce Gaussian smearings with different severity, it needs to be conditioned on the current timestep to be denoised. However, the difference between two timesteps is numerically speaking tiny, and the network may struggle to pick up the small differences between timesteps. 

Therefore we make use of Sinusoidal embedding. This translates small changes in the timestep into unique combinations of sin and cosine evaluations, based on the given timestep. 

In [ ]:
use_SinusoidalPosEmb = True

import math
def SinusoidalPosEmb(x, dim):
    device = x.device
    half_dim = dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
    emb = x[:, None] * emb[None, :]
    emb = torch.cat((emb.sin(), emb.cos()), dim=-1).view(-1,dim) 
    return emb

if use_SinusoidalPosEmb:
    embed_dim = 16
    embed_func = lambda x: SinusoidalPosEmb(x, embed_dim)
else:    
    embed_dim = 1
    embed_func = lambda x: x
    

Now we can train the diffusion model. To this end, we apply a smearing for a randomly determined timestep and then learn to undo this smearing. The loss in this case is a simple mean squared error between the applied smearing and the inverse smearing predicted by the network. 

In [ ]:
######
# Modifications here
timesteps = 10
beta = torch.linspace(start=1e-4, end=1e-1, steps=timesteps).to(device) 
######


# define alpha for forward diffusion kernel
alpha = 1 - beta
alpha_bar = torch.cumprod(alpha, dim=0)


######
# Modifications here
DenoiseModel = DenseNet(input_dim=2+embed_dim, output_dim=2, hidden_dim=256, n_layers=4).to(device)

DenoiseOpt = optim.Adam(DenoiseModel.parameters(), lr=1e-4)

batch_size = 200
epochs = 10
######


max_batches = int(X_moons.shape[0] / batch_size)

MSEloss = nn.MSELoss()
for ep in range(epochs):
    ep_loss = 0
    for i_batch in range(max_batches):
        # select the current batch from the dataset
        sample = X_moons[i_batch * batch_size : (i_batch + 1) * batch_size]
        sample = torch.tensor(sample, device=device).float()

        DenoiseOpt.zero_grad()
        
        step = np.random.randint(0, timesteps)
        step_network_input = torch.ones(sample.shape[0], 1, device=device)*step
        step_network_input = embed_func(step_network_input)
        
        
        noise = torch.randn_like(sample) 

        noisy_sample = (sample * torch.sqrt(alpha_bar[step]) + 
                        noise * torch.sqrt(1-alpha_bar[step]))

        noise_pred = DenoiseModel(torch.cat([noisy_sample, step_network_input], -1))
        
        loss = MSEloss(noise, noise_pred)
        ep_loss += loss.item()
        
        loss.backward()
        DenoiseOpt.step()

    if ep % 10 == 0:
        print(ep)
        print(ep_loss)


Finally, we apply the model to a random gaussian for each timestep to achieve a two-moons-like sample. 

In [ ]:
n_sample = 2000

noisy_sample = torch.randn((n_sample, X_moons.shape[-1])).to(device)

for t in range(timesteps-1, -1, -1):
    
    if t > 1:
        z = torch.randn_like(noisy_sample).to(device)
    elif t==0:
        z = torch.zeros_like(noisy_sample).to(device)
    
    with torch.no_grad():
        step_network_input = torch.ones(noisy_sample.shape[0], 1, device=device)*t
        step_network_input = embed_func(step_network_input)

        noise_pred = DenoiseModel(torch.cat([noisy_sample, step_network_input], -1))

    # denoise at time t, utilizing predicted noise
    noisy_sample_new = ((1 / torch.sqrt(alpha[t])) * 
                        (noisy_sample - (1-alpha[t])/torch.sqrt(1-alpha_bar[t])*noise_pred) + 
                        torch.sqrt(beta[t])*z)

    noisy_sample = noisy_sample_new
    

samples = noisy_sample
samples = samples.cpu().numpy()

# plot the results
plt.scatter(samples[:, 0], samples[:, 1], color='darkblue', marker='o', linewidth=0)
plt.scatter(X_moons[:, 0], X_moons[:, 1], color='black', marker='.', linewidth=0)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.savefig("./sampled_two_moons.pdf")
plt.show()
plt.close()

**Task: Improve the performance of the network. Tip: Modify the noise schedule. Consider using more timesteps and maybe even a different, non-linear shape**


**Bonus Task:
Modify the sampling code above to show the intermediated generation steps**

# 3 Normalizing Flows

## 3a.) Motivation

Just as Variational Autoencoders (VAEs) or Generative Adversarial Networks (GANs), Normalizing Flows are **Probabilistic Generative Models (PGMs)**, which describe a probability distribution that we attempt to learn from a set of observed data. 

PGMs are useful for generating new samples from the learned distribution, evaluating the likelihood of new data points, etc. 

Normalizing flows are PGMs built on **invertible transformations**. Their advantages are that it is typically possible to efficiently **sample** and **evaluate** the learned distributions. Normalizing flows also are **highly expressive** and come with a **useful latent space representation**, since we have a one-to-one mapping between our input and the latent space. Finally, they are also **easy to train**, since we just need to conduct a simple maximum likelihood training.

In this hands-on exercise, we will show examples of using normalizing flows to estimate (conditional) probability densities and generate new samples. While some instructive mock datasets will be used for getting to know Normalizing Flows, we will also have one dedicated exercise that will use a Particle Physics dataset in the end. As a framework for actually implementing Normalizing Flows quickly, we will use the **nFlows** package. We hope this tutorial will give you at least a rough idea of how Normalizing Flows work in practice and - most importantly - that you find it instructive and fun :)

## 3b.) Normalizing Flows - The general idea (recap)

At the heart of normalizing flows stands the change of variables formula. Let's say we had a random variable $U$ and now, we apply a simple transformation, defining the transformed random variable as $X$. Then, the change of variable formula is given by:

$$ p(x) = p(u)\left| \frac{df(u)}{du}\right|^{-1} $$

As an example, let's have a look at the figure below:

<br />
<img src="./FlowImage1.png" width="250">
<br />

We start with a uniform distribution $U$ between 0 and 1, and then do a transformation $f(U)=2\cdot U + 1$. This, however, leads to a distribution that is not normalized, so to get a proper probability distribution, we still need to scale it with $ \left| \left(\frac{df(u)}{du}\right)\right|^{-1} $.

The derivative of $f$ with respect to $u$ is 2, and the inverse is $\frac{1}{2}$, which is exactly the scaling factor that we need for getting a properly normalized distribution.

This, however, is only a simple univariate example. For the multivariate case, instead of the derivative, we need to scale our transformed distribution with the Jacobian determinant:

$$ \left|\det \left( \frac{\partial \boldsymbol{f}}{\partial \boldsymbol{u}} \right)  \right|^{-1} = \left|\det \left( \frac{\partial \boldsymbol{f}^{-1}}{\partial \boldsymbol{x}} \right)  \right| $$

The main idea of normalizing flows is to learn an invertible mapping between a very complex distribution (for example a distribution of a physics variable) and a very simple distribution. Let us consider the "generative" part first, where we attempt to generate new samples from a Normalizing Flow. In this case, we start with a simple distribution, for example, a standard Gaussian, and then repeatedly apply the random variable transformations to acquire a complex distribution similar to the input distribution that we want to approximate. Let's say we apply $k$ transformations $f_{1}...f_{k}$, starting out from a simple distribution $u_{0}$ and going to a complex distribution $u_{k}$, then the change of random variables formula becomes:

$$ p(\boldsymbol{u}_{k}) = p(\boldsymbol{u}_{0})\prod_{i} \left|\det \left( \frac{\partial \boldsymbol{f}_{i}}{\partial \boldsymbol{u}_{i-1}} \right)  \right|^{-1} $$


From this formula, we also see the reason why this method is called "normalizing flows". The random variable "flows" through a series of transformations while staying normalized by the scaling with the Jacobian determinant.

<br />
<img src="./FlowImage2.png" width="1000">
<br />

In a practical case, our "complex" random variables $\boldsymbol{u}_{k}$ would then approximate our input variables.

Since we choose our $\boldsymbol{f}_{i}$ to be invertible, we can also run the Normalizing Flow in the other direction, starting out with our complex input distribution and mapping this distribution to a standard Gaussian. This is the direction used for density/likelihood estimation (i.e. if you want to evaluate the likelihood of new data under the learned distribution) and also the one used for training.

For training a normalizing flow, remember that our $\boldsymbol{f}_{i}$ are actually neural networks with parameters $\theta$. We train our flow by optimizing these parameters in a negative logarithmic likelihood minimization (which is our loss function then). So we take our input samples $\boldsymbol{u}_{k}$, take it "backward" through the flow (i.e. in the direction of the blue arrows in the picture above), and then tune the parameters of our $\boldsymbol{f}_{i}$ such that the likelihood of the so transformed distribution under a standard Gaussian gets maximized.

To be able to use normalizing flows efficiently, we, therefore, need functions $f$ that are both **invertible** and **have a tractable Jacobian** that is **easy to compute**. Several methods have been proposed so far and today, you will work mainly with so-called "autoregressive flows", where the functions $f$ are chosen in a way such that the Jacobian is guaranteed to be an upper triangular matrix, so we can compute the determinant simply by multiplying its diagonal elements.


## 3c.) The nFlows package 

Today, we are going to use the PyTorch nFlows library, which is easy to set up and already contains implementations of many of the different flow models. 

Details and code of the package can be found here https://github.com/bayesiains/nflows

In [ ]:
# the nflows functions what we will need in order to build our flow
from nflows.flows.base import Flow  # a container that will wrap the parts that make up a normalizing flow
from nflows.distributions.normal import StandardNormal  # Gaussian latent space distribution
from nflows.transforms.base import (
    CompositeTransform,
)  # a wrapper to stack simpler transformations to form a more complex one
from nflows.transforms.autoregressive import (
    MaskedAffineAutoregressiveTransform,
)  # the basic transformation, which we will stack several times
from nflows.transforms.autoregressive import (
    MaskedPiecewiseRationalQuadraticAutoregressiveTransform,
)  # the basic transformation, which we will stack several times
from nflows.transforms.permutations import ReversePermutation  # a layer that simply reverts the order of outputs

Some background about the different modules:

The **flows module** contains the implementation of the actual flow models. The classes in this module already implement the functions to calculate the negative log-likelihood and the jacobian determinant.

Every flow model is defined as a series of transformations, contained in the **transformation module**. Here we will be using two different transformations, `MaskedAffineAutoregressiveTransform`, which provides the actual invertible transformation, and `ReversePermutation` which serves to permute the features within the flow. The final transformation `CompositeTransform` allows for the combining of several transformations into one module. 

Finally, the **distributions module** contains the necessary base distributions that form the latent space. In our case, we will use a gaussian latent space, defined by `StandardNormal`.





The first model we will want to look at is the Masked Autoregressive Flow (MAF) https://arxiv.org/abs/1705.07057.

The autoregressive property on a sequence of data $\boldsymbol{x} = [x_1, …, x_D]$ states that each output only depends on previous data. This means that the probability of $x_i$ is conditioned only on $x_1, ... ,x_{i-1}$. The full probability density $p(\boldsymbol{x})$ then becomes the product of the conditional densities:
$$p(\boldsymbol{x}) = \Pi_{i=1}^{D}p(x_i|x_1, …, x_{i-1})$$

An autoregressive flow makes use of this property in arranging the dependencies of the input dimensions. In each layer, the output $f_{i}(\boldsymbol{x})$ only depends on features $x_1, .. ,x_{i-1}$. The reason for this is the simpler form of the jacobian determinant. The matrix of partial derivates $\frac{\partial \boldsymbol{f}_{i}}{\partial \boldsymbol{x}_{j}}$ becomes triangular because there is no dependency between earlier input dimensions $\frac{\partial \boldsymbol{f}_{i}}{\partial \boldsymbol{x}_{j > i}} = 0$. As a result, the determinant is simply the product of diagonal elements.

The variable transformation of the MAF is a shift and scaling operation (i.e. an affine transformation):
$$z_i = x_i \exp(s_i(\boldsymbol{x})) + t_i(\boldsymbol{x})$$ 
but it chooses neural networks $\boldsymbol{s}$ and $\boldsymbol{t}$ such that they respect the autoregressive property $s_i(x_1, .., x_{i-1})$ and $t_i(x_1, .., x_{i-1})$. It achieves this by stacking so-called MADE blocks (https://arxiv.org/abs/1502.03509), which are sets of neural network layers that apply element-wise multiplications of mask matrices to the weights. The mask matrices simply contain ones and zeros, intending to turn off dependencies that would violate the autoregressive property. After each flow layer, one applies a permutation on the order of inputs.


Let's now define such a simple MAF with the nflows package. In this framework, a flow consists of a base (latent space) distribution and an invertible transformation. This flow object then has two important methods: `flow.log_prob(data)` returns the logarithmic probability of the data, `flow.sample(n_samples)` produces new data points.


In [ ]:
# Fist we define the latent space distribution, in this case a choosing a 2-dim Gaussian
base_dist = StandardNormal(shape=[2])

# Now we define the series of transformation that our flow model will comprise.
# For now we will use a singel MAF layer, we preemtivly implement this as a list,
# so to make it simple to extend.

# For the MAF layer we need to secify the number of input parameters, as well as the
# number of features used in the internal FCN layers.
transforms = []
transforms.append(MaskedAffineAutoregressiveTransform(features=2, hidden_features=4))

# Finally we can combine the list of previously defined transformations into one
# single compisite transformation.
transform = CompositeTransform(transforms)

# The actual flow now consist of the the base distribution and transform together
flow = Flow(transform, base_dist).to(device)

# we can then use standard PyTorch optimizers
optimizer = optim.Adam(flow.parameters())

# 3d.) Data Generation Using Flows

One major advantage of Normalizing Flow Networks is that they are inherently invertible. This means that we can not only use them for density estimation by mapping our data set to a well-known distribution like a Normal Gaussian, but we can also do the reverse, mapping the Normal Gaussian to our data distribution. This essentially allows us to generate new data by feeding Gaussian samples backward through our model. 





In [ ]:
batch_size = 200

max_batches = int(X_moons.shape[0] / batch_size)

for i_batch in range(max_batches):
    # select the current batch from the dataset
    x = X_moons[i_batch * batch_size : (i_batch + 1) * batch_size]
    x = torch.tensor(x, device=device).float()
    optimizer.zero_grad()

    # calculate negative log likelihood
    nll = -flow.log_prob(x)

    # update the model
    loss = nll.mean()
    loss.backward()
    optimizer.step()

In [ ]:
# Let's use the fitted distribution and sample from it
with torch.no_grad():
    samples = flow.sample(1000).cpu().numpy()


# plot the results
plt.scatter(samples[:, 0], samples[:, 1], color='darkblue', marker='o', linewidth=0)
plt.scatter(X_moons[:, 0], X_moons[:, 1], color='black', marker='.', linewidth=0)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.savefig("./sampled_two_moons.pdf")
plt.show()
plt.close()

In [ ]:
# Let's also plot how well our data is mapped to the Gaussian base distribution

inv = flow.transform_to_noise(torch.tensor(X_moons).float().to(device)).detach().cpu().numpy()
plt.scatter(inv[:, 0], inv[:, 1], color='black', marker='.', linewidth=0)
plt.xlabel("Transformed x1")
plt.ylabel("Transformed x2")
plt.show()
plt.close()

As we can see, our initial density is estimated rather poorly. We also see that our mapped data is far from being Gaussian distributed in the latent space. So what did we do wrong? We actually didn't use a proper flow! For getting to know nFLows, a single MAF transformation was used here. With the knowledge of how to use the module, we can now start building a powerful flow, consisting of multiple layers.

**Task: Modify the Code below to improve the generation result**

**Bonus Task: Rational Quadratic Splines (RQS) are a more complex and advanced type of invertible transformations, that see use in state-of-the-art normalizing flows. nFlows provides a prebuilt implementation of these transformations, which can be accessed via:**

```
MaskedPiecewiseRationalQuadraticAutoregressiveTransform(features=2, 
hidden_features=16, tail_bound = 3.0, tails = "linear"))
```

**Try replacing the MAF transformations with RQS transformations and see what effects can be observed observe.** 


# 3e.) Two Moons Generation




In [ ]:
base_dist = StandardNormal(shape=[2])

######
# Modifications here
transforms = []
transforms.append(MaskedAffineAutoregressiveTransform(features=2, hidden_features=16))
transforms.append(ReversePermutation(features=2))
######


transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)

optimizer = optim.Adam(flow.parameters())


######
# Modifications here
num_epochs = 1
batch_size = 200
######

max_batches = int(X_moons.shape[0] / batch_size)

for i in range(num_epochs):
    permut = np.random.permutation(X_moons.shape[0])
    X_moons_shuffle = X_moons[permut]
    for i_batch in range(max_batches):
        x = X_moons_shuffle[i_batch * batch_size : (i_batch + 1) * batch_size]
        x = torch.tensor(x, device=device).float()
        optimizer.zero_grad()

        nll = -flow.log_prob(x)

        loss = nll.mean()
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print('Epoch: {:d}'.format(i))

with torch.no_grad():
    samples = flow.sample(1000).cpu().numpy()

# transform them like we did when plotting the original dataset
# samples = StandardScaler().fit_transform(samples)

# plot the results
plt.scatter(samples[:, 0], samples[:, 1], color='darkblue', marker='o', linewidth=0)
plt.scatter(X_moons[:, 0], X_moons[:, 1], color='black', marker='.', linewidth=0)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.savefig("./sampled_two_moons.pdf")
plt.show()
plt.close()

# 3f.) Intermediate transformation steps

So far, our implementation of normalizing flows was very effective and easy to use, however it was also rather BlackBox-y. 

The code below illustrates how one can extract the individual transformations and the base distribution from the flow model, and then perform the first `n_transforms` transformations "by hand". 

In [ ]:
# Number of transforms to perform
n_transforms = 1

# Ensure the number of transforms to perform is lower than the total amount of transforms
n_transforms = min(len(flow._transform._transforms), n_transforms)

# Here we access the base distribution of the flow model and sample from it
z = flow._distribution.sample(1000)


with torch.no_grad():
    for i in range(1, n_transforms):
        # The flow model has an attribute called '_transform', which again has an attribute
        # '_transforms', which is a list containing the individual transforms
        # For the purpose of generating samples we need to iterate through the transforms in
        # reverse order, hence the negative index
        m = flow._transform._transforms[-i]

        # The transforms can be applied like any other PyTorch module, however, we need
        # to specify the direction, in the case of sampling we need the inverse.
        # The function has two return parameters, the transformed data, and the log-determinant
        # for the sample generation we only care about the former and map the latter to a _
        z, _ = m.inverse(z)


plt.scatter(z.cpu().numpy()[:, 0], z.cpu().numpy()[:, 1], color='darkblue', marker='o', linewidth=0)
plt.savefig("./sampled_two_moons.pdf")
plt.show()
plt.close()

So far we only see the effects of the first transformation

**Task: Use the above code to plot the intermediate distributions for all flow transformations**


In [ ]:
#################
# Add code here #
#################

## 3g.) Conditional Generative Flows

Many generative tasks in particle physics require the generative model to not only generate a random point from the data set but instead generate a point with a specific property. One example is shower simulation, where a generative shower simulator should be able to produce showers for a given particle energy, rather than just for a random one. 

This is where conditional generative models become important. In the previous example, the energy value would be the condition given to the model. 

Let us try this principle on flows with our previous two-moons data set. As one can see, it is made up of two distinct subsets, (the eponymous moons). Our conditioning will be to which moon a datapoint belongs, e.g. if we tell the model to generate points with the label 0 they should be from the top moon, while points with the label 1 should be from the bottom.

In [ ]:
from sklearn import cluster, datasets, mixture
from sklearn.preprocessing import StandardScaler

# for two moons dataset
n = 10000
# the two moons sampling function already returns labels to which moon a data point belongs
X_moons, label_moon = datasets.make_moons(n_samples=n, noise=0.05)
X_moons, mask = normalize_moons(X_moons)
label_moon = label_moon[mask]


plt.scatter(X_moons[:, 0][label_moon == 0], X_moons[:, 1][label_moon == 0], color='orangered', marker='.', linewidth=0)
plt.scatter(X_moons[:, 0][label_moon == 1], X_moons[:, 1][label_moon == 1], color='green', marker='.', linewidth=0)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.savefig("./original_two_moons.pdf")
plt.show()
plt.close()

### Conditional nFlows models

Conditional models in nFlows are handled via the `context` arguments. During the definition of each transform layer, the number of conditional variables has to be specified. 

Then, during training and generation, the context has to be passed as additional input.

In [ ]:
n_features = 2
n_layers = 10
base_dist = StandardNormal(shape=[n_features])

transforms = []
for i in range(0, n_layers):
    transforms.append(
        MaskedAffineAutoregressiveTransform(features=n_features, hidden_features=16, context_features=1)
    )  # note the context_features argument
    transforms.append(ReversePermutation(features=n_features))

transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)

optimizer = optim.Adam(flow.parameters())


num_epochs = 20
batch_size = 200

max_batches = int(X_moons.shape[0] / batch_size)

for i in range(num_epochs):
    permut = np.random.permutation(X_moons.shape[0])
    X_moons_shuffle = X_moons[permut]
    label_moon_shuffle = label_moon[permut]

    for i_batch in range(max_batches):
        x = X_moons_shuffle[i_batch * batch_size : (i_batch + 1) * batch_size]
        x = torch.tensor(x, device=device).float()

        y = label_moon_shuffle[i_batch * batch_size : (i_batch + 1) * batch_size]
        y = torch.tensor(y, device=device).float().view(-1, 1)
        optimizer.zero_grad()

        nll = -flow.log_prob(x, context=y)  # note the context argument

        loss = nll.mean()
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print('Epoch: {:d}'.format(i))

Now lets see how well our model performs, first by using random labels. This should look like the default two moons set. 

In [ ]:
n_samples = 10000
cond = np.random.randint(2, size=(n_samples, 1))
cond = torch.tensor(cond).float().to(device)

with torch.no_grad():
    samples = flow.sample(1, context=cond).view(n_samples, -1).cpu().numpy()

print(samples.shape)

# plot the results
plt.scatter(samples[:, 0], samples[:, 1], color='darkblue', marker='o', linewidth=0)
plt.scatter(X_moons[:, 0], X_moons[:, 1], color='black', marker='.', linewidth=0)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.savefig("./sampled_two_moons.pdf")
plt.show()
plt.close()

We can also test how well our conditioning performs by passing only the label 0 or only label 1 to our network. 

**Task: Complete the code below to generate points from the two moons separately, and then plot the two moons in separate colors**

**Bonus task: What happens when the flow is evaluated using labels different from 0 and 1**

In [ ]:
n_samples = 10000


######
# Modifications here
samples1 = ...
samples0 = ...

plt.scatter(samples0[:, 0], samples0[:, 1], color='orangered', marker='.', linewidth=0)
plt.scatter(samples1[:, 0], samples1[:, 1], color='green', marker='.', linewidth=0)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.savefig("./sampled_two_moons.pdf")
plt.show()
plt.close()